# Electoral College
Not all Global Data Plane tables are lists of rows.  In fact, a GDP table is _any_ structure that implements the `SDMLTable` API.  In this Notebook, we show a _very_ simple example of serving rows via computation.

In [ ]:
import json
import sdtp


The Electoral College Raw table has columns (Year, Candidate, Party, Votes).  We want to generate a table with columns (Year, Republican, Democratic, Whig, Other, Not_Voted, Non_Candidate).  Here, we're showing the simple way to do it, just by computing the rows and passing them to `SDMLFixedTable` which will implement the API for us.  

In [ ]:
with open('tables/electoral_college_raw.sdml') as f:
  table_desc = json.load(f)
  electoral_college_raw = sdtp.TableBuilder.build_table(table_desc)

A little class to compute and hold the rows of the table.  It will compute the rows the first time, and then deliver them from the cache in future.  We also have a utility class to hold the data for each year

In [ ]:
class Year:
  # This class accumulates the summary record for a given year
  def __init__(self, year):
    self.year = year
    self.keys = ["Republican", "Democratic", "Whig", "Other", "Not Voted", "not a candidate"]
    self.votes = {}
    for key in self.keys: self.votes[key] = 0
  
  def update(self, ec_row):
    # Process a new row in the original table.  Just figure out what the
    # party is an add in the votes
    party = ec_row[2]
    votes = ec_row[3]
    if party in self.keys:
      pass
    elif party == "National Republican":
      party = "Whig"
    elif party == '':
      party = 'Not Voted'
    else:
      party = "Other"
    self.votes[party] += votes
  
  def report(self):
    # Return the row for this year.  Note that the order of self.keys must match the
    # order of the schema for the resulting table
    return [self.year] + [self.votes[key] for key in self.keys]

class ElectoralCollegeHistory:
  def __init__(self, ec_table):
    self.rows = None
    self.ec_table = ec_table
  
  def get_rows(self):
    if self.rows: return self.rows
    else:
      all_years = self.ec_table.all_values('Year')
      years = {}
      for year in all_years:
        years[year] = Year(year)
      for row in self.ec_table.rows:
        year_record = years[row[0]]
        year_record.update(row)
      rows = [year_record.report() for year_record in years.values()]
      self.rows = rows
      return rows

Form the helper object

In [ ]:
ec_history = ElectoralCollegeHistory(electoral_college_raw)


Create the history table

In [ ]:
column_names = ["Year", "Republican", "Democratic", "Whig", "Other", "Not_Voted", "Non_Candidate"]
schema = [{"name": column_name, "type": "number"} for column_name in column_names]
history_table = sdtp.SDMLFixedTable(schema, ec_history.get_rows)

And now it can be queried like any other table

In [ ]:
lower = sdtp.GE("Year", 1828)
upper = sdtp.LE("Year", 1848)

history_table.get_filtered_rows(sdtp.ALL(lower, upper))